In [1]:
using Arrow
using CSV
using DataFrames
using Statistics

In [2]:
url = ("https://archive.ics.uci.edu/ml/machine-learning-databases/"
       * "auto-mpg/auto-mpg.data-original")
download(url, "auto.txt")
readlines("auto.txt")

406-element Vector{String}:
 "18.0   8.   307.0      130.0      3504.      12.0   70.  1.\t\"chevrolet chevelle malibu\""
 "15.0   8.   350.0      165.0      3693.      11.5   70.  1.\t\"buick skylark 320\""
 "18.0   8.   318.0      150.0      3436.      11.0   70.  1.\t\"plymouth satellite\""
 "16.0   8.   304.0      150.0      3433.      12.0   70.  1.\t\"amc rebel sst\""
 "17.0   8.   302.0      140.0      3449.      10.5   70.  1.\t\"ford torino\""
 "15.0   8.   429.0      198.0      4341.      10.0   70.  1.\t\"ford galaxie 500\""
 "14.0   8.   454.0      220.0      4354.       9.0   70.  1.\t\"chevrolet impala\""
 "14.0   8.   440.0      215.0      4312.       8.5   70.  1.\t\"plymouth fury iii\""
 "14.0   8.   455.0      225.0      4425.      10.0   70.  1.\t\"pontiac catalina\""
 "15.0   8.   390.0      190.0      3850.       8.5   70.  1.\t\"amc ambassador dpl\""
 "NA     4.   133.0      115.0      3090.      17.5   70.  2.\t\"citroen ds-21 pallas\""
 "NA     8.   350.0      1

In [3]:
rawstr = read("auto.txt", String);
strnotab = replace(rawstr, '\t' => ' ')
io = IOBuffer(strnotab)

IOBuffer(data=UInt8[...], readable=true, writable=false, seekable=true, append=false, size=32149, maxsize=Inf, ptr=1, mark=-1)

In [4]:
fields = [:mpg, :cylinders, :displacement, :horsepower, :weight, 
          :acceleration, :year, :origin, :name]
df1 = CSV.File(
    io,
    delim=' ',
    ignorerepeated=true,
    header=fields,
    missingstring="NA"
) |> DataFrame

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0


In [5]:
ENV["COLUMNS"], ENV["LINES"] = 200, 10
df1

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina


In [6]:
dfraw = CSV.File("auto.txt", header=[:metrics, :name]) |> DataFrame

,metrics,name
,String,String
1,18.0 8. 307.0 130.0 3504. 12.0 70. 1.,chevrolet chevelle malibu
2,15.0 8. 350.0 165.0 3693. 11.5 70. 1.,buick skylark 320
3,18.0 8. 318.0 150.0 3436. 11.0 70. 1.,plymouth satellite
4,16.0 8. 304.0 150.0 3433. 12.0 70. 1.,amc rebel sst
5,17.0 8. 302.0 140.0 3449. 10.5 70. 1.,ford torino
6,15.0 8. 429.0 198.0 4341. 10.0 70. 1.,ford galaxie 500
7,14.0 8. 454.0 220.0 4354. 9.0 70. 1.,chevrolet impala
8,14.0 8. 440.0 215.0 4312. 8.5 70. 1.,plymouth fury iii
9,14.0 8. 455.0 225.0 4425. 10.0 70. 1.,pontiac catalina


In [7]:
strmetrics = split.(dfraw.metrics)

406-element Vector{Vector{SubString{String}}}:
 ["18.0", "8.", "307.0", "130.0", "3504.", "12.0", "70.", "1."]
 ["15.0", "8.", "350.0", "165.0", "3693.", "11.5", "70.", "1."]
 ["18.0", "8.", "318.0", "150.0", "3436.", "11.0", "70.", "1."]
 ⋮
 ["28.0", "4.", "120.0", "79.00", "2625.", "18.6", "82.", "1."]
 ["31.0", "4.", "119.0", "82.00", "2720.", "19.4", "82.", "1."]

In [8]:
df12 = DataFrame([col => Float64[] for col in fields[1:(end - 1)]])

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [9]:
allowmissing!(df12, [:mpg, :horsepower])

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64


In [10]:
for row in strmetrics
    push!(df12, [v == "NA" ? missing : parse(Float64, v) for v in row])
end

In [11]:
df12

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0


In [12]:
df12.name = dfraw.name
df12

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina


In [13]:
df12.name === dfraw.name #!

true

In [14]:
isequal(df12, df1)

true

In [16]:
df12 == df1

missing

In [17]:
df13 = select(
    dfraw,
    :metrics => 
        ByRow(x -> something.(tryparse.(Float64, split(x)), missing)) =>
        fields[1:(end - 1)],
    :name)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina


In [18]:
isequal(df13, df1)

true

In [19]:
sum(count(ismissing, col) for col in eachcol(df1))

14

In [21]:
count(ismissing, Matrix(df1))

14

In [22]:
count(ismissing, Iterators.flatten(eachcol(df1)))

14

In [23]:
mapcols(x -> count(ismissing, x), df1)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,8,0,0,6,0,0,0,0,0


In [24]:
filter(row -> any(ismissing, row), df1)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String
1,missing,4.0,133.0,115.0,3090.0,17.5,70.0,2.0,citroen ds-21 pallas
2,missing,8.0,350.0,165.0,4142.0,11.5,70.0,1.0,chevrolet chevelle concours (sw)
3,missing,8.0,351.0,153.0,4034.0,11.0,70.0,1.0,ford torino (sw)
4,missing,8.0,383.0,175.0,4166.0,10.5,70.0,1.0,plymouth satellite (sw)
5,missing,8.0,360.0,175.0,3850.0,11.0,70.0,1.0,amc rebel sst (sw)
6,missing,8.0,302.0,140.0,3353.0,8.0,70.0,1.0,ford mustang boss 302
7,25.0,4.0,98.0,missing,2046.0,19.0,71.0,1.0,ford pinto
8,missing,4.0,97.0,48.0,1978.0,20.0,71.0,2.0,volkswagen super beetle 117
9,21.0,6.0,200.0,missing,2875.0,17.0,74.0,1.0,ford maverick


In [26]:
df1.brand = first.(split.(df1.name))
df1

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64?,Float64,Float64,Float64?,Float64,Float64,Float64,Float64,String,SubStri…
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320,buick
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite,plymouth
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst,amc
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500,ford
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala,chevrolet
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii,plymouth
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina,pontiac


In [27]:
df2 = dropmissing(df1)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,SubStri…
1,18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet
2,15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320,buick
3,18.0,8.0,318.0,150.0,3436.0,11.0,70.0,1.0,plymouth satellite,plymouth
4,16.0,8.0,304.0,150.0,3433.0,12.0,70.0,1.0,amc rebel sst,amc
5,17.0,8.0,302.0,140.0,3449.0,10.5,70.0,1.0,ford torino,ford
6,15.0,8.0,429.0,198.0,4341.0,10.0,70.0,1.0,ford galaxie 500,ford
7,14.0,8.0,454.0,220.0,4354.0,9.0,70.0,1.0,chevrolet impala,chevrolet
8,14.0,8.0,440.0,215.0,4312.0,8.5,70.0,1.0,plymouth fury iii,plymouth
9,14.0,8.0,455.0,225.0,4425.0,10.0,70.0,1.0,pontiac catalina,pontiac


In [28]:
df2[df2.brand .== "saab", :]

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,SubStri…
1,25.0,4.0,104.0,95.0,2375.0,17.5,70.0,2.0,saab 99e,saab
2,24.0,4.0,121.0,110.0,2660.0,14.0,73.0,2.0,saab 99le,saab
3,25.0,4.0,121.0,115.0,2671.0,13.5,75.0,2.0,saab 99le,saab
4,21.6,4.0,121.0,115.0,2795.0,15.7,78.0,2.0,saab 99gle,saab


In [29]:
filter(:brand => ==("saab"), df2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,SubStri…
1,25.0,4.0,104.0,95.0,2375.0,17.5,70.0,2.0,saab 99e,saab
2,24.0,4.0,121.0,110.0,2660.0,14.0,73.0,2.0,saab 99le,saab
3,25.0,4.0,121.0,115.0,2671.0,13.5,75.0,2.0,saab 99le,saab
4,21.6,4.0,121.0,115.0,2795.0,15.7,78.0,2.0,saab 99gle,saab


In [30]:
filter(row -> row.brand == "saab", df2)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,SubStri…
1,25.0,4.0,104.0,95.0,2375.0,17.5,70.0,2.0,saab 99e,saab
2,24.0,4.0,121.0,110.0,2660.0,14.0,73.0,2.0,saab 99le,saab
3,25.0,4.0,121.0,115.0,2671.0,13.5,75.0,2.0,saab 99le,saab
4,21.6,4.0,121.0,115.0,2795.0,15.7,78.0,2.0,saab 99gle,saab


In [32]:
DATA = "../../data"

"../../data"

In [35]:
CSV.write("$DATA/auto2.csv", df2)

"../../data/auto2.csv"

In [36]:
Arrow.write("$DATA/auto2.arrow", df2)

"../../data/auto2.arrow"

In [37]:
readlines("$DATA/auto2.csv")

393-element Vector{String}:
 "mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,brand"
 "18.0,8.0,307.0,130.0,3504.0,12.0,70.0,1.0,chevrolet chevelle malibu,chevrolet"
 "15.0,8.0,350.0,165.0,3693.0,11.5,70.0,1.0,buick skylark 320,buick"
 ⋮
 "28.0,4.0,120.0,79.0,2625.0,18.6,82.0,1.0,ford ranger,ford"
 "31.0,4.0,119.0,82.0,2720.0,19.4,82.0,1.0,chevy s-10,chevy"